# Beam Simulations
Getting started with notes from James Lamb.

Let me start with 1.35 GHz.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from numpy import fft

In [2]:
coeffs = np.array([0.41014, 0.09540, -0.27752, 0.17076, -0.05474, 0.00988, -0.00095])

In [3]:
EXTENT_M = 250
GRID_SIZE = 1000
LAMBD = 3e8/1.35e9

In [4]:
grid_cell = EXTENT_M / (GRID_SIZE//2)

In [5]:
ea_grid_m = np.meshgrid(np.linspace(-EXTENT_M, EXTENT_M, num=GRID_SIZE + 1), np.linspace(-EXTENT_M, EXTENT_M, num=GRID_SIZE + 1))
ea_grid = np.sum(np.power(np.sqrt(ea_grid_m[0][:-1, :-1]**2 + ea_grid_m[1][:-1, :-1]**2), np.arange(1, len(coeffs) + 1)[:,np.newaxis, np.newaxis]), axis=0)
ea_grid[np.sqrt(ea_grid_m[0][:-1,:-1]**2 + ea_grid_m[1][:-1, :-1]**2) > 5] = 0

In [23]:
%matplotlib widget
fig, ax = plt.subplots()
ax.set_aspect(1)
p = ax.pcolor(ea_grid_m[0], ea_grid_m[1], np.log10(ea_grid))
ax.set_xlabel('X (m)')
ax.set_ylabel('y (m)')
ax.set_title('Antenna Illumination electric field (Ea)')
fig.colorbar(p)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<ipython-input-23-bf2664f5b7b3>:4: RuntimeWarning: divide by zero encountered in log10
  p = ax.pcolor(ea_grid_m[0], ea_grid_m[1], np.log10(ea_grid))


In [7]:
e_beam = fft.fft2(ea_grid)

In [8]:
e_beam = fft.fftshift(e_beam)

In [9]:
freqs = fft.fftfreq(GRID_SIZE, grid_cell/(2 * np.pi * LAMBD))

In [10]:
ft_grid = np.meshgrid(freqs, freqs)

In [11]:
amp = np.abs(e_beam)

In [12]:
mfreq = freqs.max() * 57.2958

In [13]:
d1 = np.argsort(freqs)
d2 = np.argsort(freqs)

In [24]:
%matplotlib widget
plt.imshow(np.abs(e_beam), extent=[-mfreq, mfreq, -mfreq, mfreq])
ax.set_xlabel('deg')
ax.set_ylabel('deg')
ax.set_title('Far-field electric field (E)')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'Far-field electric field (E)')

In [16]:
from scipy import optimize
def gaussian(height, center_x, center_y, width_x, width_y):
    """Returns a gaussian function with the given parameters"""
    width_x = float(width_x)
    width_y = float(width_y)
    return lambda x,y: height*np.exp(
                -(((center_x-x)/width_x)**2+((center_y-y)/width_y)**2)/2)

def moments(data):
    """Returns (height, x, y, width_x, width_y)
    the gaussian parameters of a 2D distribution by calculating its
    moments """
    total = data.sum()
    X, Y = np.indices(data.shape)
    x = (X*data).sum()/total
    y = (Y*data).sum()/total
    col = data[:, int(y)]
    width_x = np.sqrt(np.abs((np.arange(col.size)-x)**2*col).sum()/col.sum())
    row = data[int(x), :]
    width_y = np.sqrt(np.abs((np.arange(row.size)-y)**2*row).sum()/row.sum())
    height = data.max()
    return height, x, y, width_x, width_y

def fitgaussian(data):
    """Returns (height, x, y, width_x, width_y)
    the gaussian parameters of a 2D distribution found by a fit"""
    params = moments(data)
    errorfunction = lambda p: np.ravel(gaussian(*p)(*np.indices(data.shape)) -
                                 data)
    p, success = optimize.leastsq(errorfunction, params)
    return p

In [17]:
data = amp[d1][:,d2][450:550, 450:550] ** 2

In [18]:
params = fitgaussian(data)

In [19]:
params

array([3.76035193e+11, 4.99989148e+01, 4.99989148e+01, 1.37405372e+01,
       1.37405372e+01])

In [20]:
amp.shape

(1000, 1000)

In [21]:
%matplotlib widget
fit = gaussian(*params)
plt.imshow(data)
plt.contour(fit(*np.indices(data.shape)), cmap=plt.cm.copper)
plt.colorbar()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [22]:
params[3] * freqs[1] * 57.2958

2.198486737514539